In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt

from augmentation import rotate
from keras_scope.datasets.gsd_outcome_dataset import get_gsd_outcome_dataset
from keras_scope.datasets.leftright_dataset import get_LeftRightDataset
from model import get_model
from preprocessing import min_max_normalize, resize_volume



In [ ]:
def plot_slices(num_rows, num_columns, width, height, data):
    """Plot a montage of 20 CT slices"""
    data = np.rot90(np.array(data))
    data = np.transpose(data)
    data = np.reshape(data, (num_rows, num_columns, width, height))
    rows_data, columns_data = data.shape[0], data.shape[1]
    heights = [slc[0].shape[0] for slc in data]
    widths = [slc.shape[1] for slc in data[0]]
    fig_width = 12.0
    fig_height = fig_width * sum(heights) / sum(widths)
    f, axarr = plt.subplots(
        rows_data,
        columns_data,
        figsize=(fig_width, fig_height),
        gridspec_kw={"height_ratios": heights},
    )
    for i in range(rows_data):
        for j in range(columns_data):
            axarr[i, j].imshow(data[i][j], cmap="gray")
            axarr[i, j].axis("off")
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()

In [ ]:
label_file_path = '/Users/jk1/stroke_research/SimpleVoxel-3D/leftright/labels.csv'
data_file_path = '/Users/jk1/stroke_research/SimpleVoxel-3D/leftright/data.npy'
desired_shape = (46, 46, 46)
split_ratio = 0.3
batch_size = 2
train_dataset, validation_dataset = get_LeftRightDataset(label_file_path, data_file_path, desired_shape, split_ratio, batch_size)

data = train_dataset.take(1)
images, labels = list(data)[0]
images = images.numpy()
image = images[0]
# Visualize montage of slices.
# 4 rows and 10 columns for 100 slices of the CT scan.
plot_slices(4, 10, 46, 46, image[:, :, :40])


In [ ]:
label_file_path = '/Users/jk1/temp/scope_test/joined_anon_outcomes_2015_2016_2017_2018_df.xlsx'
imaging_dataset_path = '/Users/jk1/temp/scope_test/subset5_noGT_pmaps_15-19_dataset_with_combined_mRS_90_days.npz'
main_log_dir = '/Users/jk1/temp/keras_scope_test'
channels = [0, 1, 2, 3]
outcome = "combined_mRS_0-2_90_days"
desired_shape = (46, 46, 46)
split_ratio = 0.3
batch_size = 2
train_dataset, validation_dataset = get_gsd_outcome_dataset(label_file_path, imaging_dataset_path,
                                                                outcome, channels,
                                                                desired_shape, split_ratio, batch_size)


In [ ]:
data = train_dataset.take(1)
images, labels = list(data)[0]
images = images.numpy()
image = images[0]
# Visualize montage of slices.
# 4 rows and 10 columns for 100 slices of the CT scan.
plot_slices(4, 10, 46, 46, image[:, :, :40, 3])